# Ratings_cleaning musi zostać uruchomiony jako pierwszy
Notebook `Ratings_cleaning.ipynb` usuwa niektóre z wystawionych ocen, co zmniejsza ich liczbę u użytkowników. Następnie usuwa użytkowników, którzy mają mniej niż 20 ocen i zapisuje nowy zbiór danych do pliku `user_features_2.csv`.

In [51]:
import pandas as pd

In [52]:
df = pd.read_csv('../datasets/user_features_2.csv')

In [53]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200846 entries, 0 to 200845
Data columns (total 24 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   userId                 200846 non-null  int64  
 1   num_rating             200846 non-null  int64  
 2   avg_rating             200846 non-null  float64
 3   weekend_watcher        200846 non-null  int64  
 4   type_of_viewer         200846 non-null  object 
 5   genre_Action           199613 non-null  float64
 6   genre_Adventure        199878 non-null  float64
 7   genre_Animation        173940 non-null  float64
 8   genre_Comedy           200378 non-null  float64
 9   genre_Crime            198682 non-null  float64
 10  genre_Documentary      72068 non-null   float64
 11  genre_Drama            200781 non-null  float64
 12  genre_Family           188451 non-null  float64
 13  genre_Fantasy          196283 non-null  float64
 14  genre_History          173911 non-nu

# Uzupełnienie brakujących ocen 

In [54]:
genre_columns = ['genre_Action','genre_Adventure','genre_Animation','genre_Comedy','genre_Crime','genre_Documentary','genre_Drama','genre_Family','genre_Fantasy','genre_History','genre_Horror','genre_Music','genre_Mystery','genre_Romance','genre_Science Fiction','genre_TV Movie','genre_Thriller','genre_War','genre_Western']

for col in genre_columns:
    df[col] = df[col].fillna(df['avg_rating'])

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200846 entries, 0 to 200845
Data columns (total 24 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   userId                 200846 non-null  int64  
 1   num_rating             200846 non-null  int64  
 2   avg_rating             200846 non-null  float64
 3   weekend_watcher        200846 non-null  int64  
 4   type_of_viewer         200846 non-null  object 
 5   genre_Action           200846 non-null  float64
 6   genre_Adventure        200846 non-null  float64
 7   genre_Animation        200846 non-null  float64
 8   genre_Comedy           200846 non-null  float64
 9   genre_Crime            200846 non-null  float64
 10  genre_Documentary      200846 non-null  float64
 11  genre_Drama            200846 non-null  float64
 12  genre_Family           200846 non-null  float64
 13  genre_Fantasy          200846 non-null  float64
 14  genre_History          200846 non-nu

# Enkodowanie kolumny 'type_of_viewer'

In [55]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse_output=False)
encoded = ohe.fit_transform(df[['type_of_viewer']])
encoded = pd.DataFrame(encoded, columns=ohe.get_feature_names_out(['type_of_viewer']))

encoded.index = df.index
df = df.drop(columns=['type_of_viewer'])
df = pd.concat([df, encoded], axis=1)

print(df.info())
print()
print(df.head(3))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200846 entries, 0 to 200845
Data columns (total 26 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   userId                   200846 non-null  int64  
 1   num_rating               200846 non-null  int64  
 2   avg_rating               200846 non-null  float64
 3   weekend_watcher          200846 non-null  int64  
 4   genre_Action             200846 non-null  float64
 5   genre_Adventure          200846 non-null  float64
 6   genre_Animation          200846 non-null  float64
 7   genre_Comedy             200846 non-null  float64
 8   genre_Crime              200846 non-null  float64
 9   genre_Documentary        200846 non-null  float64
 10  genre_Drama              200846 non-null  float64
 11  genre_Family             200846 non-null  float64
 12  genre_Fantasy            200846 non-null  float64
 13  genre_History            200846 non-null  float64
 14  genr

# Dołączenie ostatnich 20 ocenionych filmów ze zbioru ratings_clean_zlib.h5

In [56]:
ratings_df = pd.read_parquet('../datasets/ratings_clean.parquet')
df = pd.merge(df, ratings_df, on='userId')

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200846 entries, 0 to 200845
Data columns (total 29 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   userId                   200846 non-null  int64  
 1   num_rating               200846 non-null  int64  
 2   avg_rating               200846 non-null  float64
 3   weekend_watcher          200846 non-null  int64  
 4   genre_Action             200846 non-null  float64
 5   genre_Adventure          200846 non-null  float64
 6   genre_Animation          200846 non-null  float64
 7   genre_Comedy             200846 non-null  float64
 8   genre_Crime              200846 non-null  float64
 9   genre_Documentary        200846 non-null  float64
 10  genre_Drama              200846 non-null  float64
 11  genre_Family             200846 non-null  float64
 12  genre_Fantasy            200846 non-null  float64
 13  genre_History            200846 non-null  float64
 14  genr

In [57]:
df.to_parquet('../datasets/user_features_clean.parquet', compression='brotli')